In [2]:
import sys
!{sys.executable} -m pip install pycbc ligo-common --no-cache-dir

     |████████████████████████████████| 6.7 MB 4.7 MB/s 
     |████████████████████████████████| 45 kB 34.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 54.3 MB/s 
     |████████████████████████████████| 51 kB 52.0 MB/s 
     |████████████████████████████████| 7.3 MB 32.0 MB/s 
     |████████████████████████████████| 201 kB 67.6 MB/s 
     |████████████████████████████████| 48.3 MB 1.2 MB/s 
     |████████████████████████████████| 45 kB 45.2 MB/s 
     |████████████████████████████████| 1.4 MB 51.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 55 kB 40.5 MB/s 
     |████████████████████████████████| 295 kB 42.1 MB/s 
     |████████████████████████████████| 43 kB 29.3 MB/s 
  Installing build dependencies ... done
  Getting requirem

In [5]:
from pycbc.waveform import get_td_waveform
from pycbc.filter import match, sigma
from pycbc.psd import aLIGOZeroDetHighPower
from pycbc.psd import aLIGOaLIGO140MpcT1800545
from pycbc.psd import aLIGOQuantumHighFrequency
from pycbc.psd import aLIGOaLIGOO3LowT1800545
from pycbc.psd import aLIGOMidHighSensitivityP1200087
from pycbc.psd import aLIGOaLIGO175MpcT1800545
from pycbc.psd import EinsteinTelescopeP1600143
from pycbc.psd import aLIGOLateLowSensitivityP1200087

#we need to import the noise curves and we do so from lalsimulation

spectrums=[aLIGOaLIGOO3LowT1800545,aLIGOaLIGO140MpcT1800545,aLIGOaLIGO175MpcT1800545,EinsteinTelescopeP1600143]

f_low = 30

d=300        #set deformation for our deformed waveform
m1=1.36      #mass1 = mass2
m2=1.36
sample_rate = 4096
# Generate the two waveforms to compare, make sure the parameters are the same between each one except for lambda
hp, hc = get_td_waveform(approximant="TaylorF2",
                         mass1=m1,                    
                         mass2=m2,
                         f_lower=f_low,
                         delta_t=1.0/sample_rate,
                         distance=40,
                         inclination=149)

sp, sc = get_td_waveform(approximant="TaylorF2",
                         lambda1=d,
                         lambda2=d,
                         mass1=m1,
                         mass2=m2,
                         f_lower=f_low,
                         delta_t=1.0/sample_rate,
                         distance=40,
                         inclination=149)

for spectrum in spectrums:
    #when computed for equal masses, the deformability parameter
    #for each object is 1.2345 times that of the "total" 



    # Resize the waveforms to the same length
    tlen = max(len(sp), len(hp))         #determine the longest wavelength in terms of time
    sp.resize(tlen)                      #ressize both signals to that of the longest so that no data is lost
    hp.resize(tlen)                      #ditto 
    lambda1=d                            #deformation = d (as before)
    lambda2=d                            #ditto

    # Generate the aLIGO ZDHP PSD
    delta_f = 1.0 / sp.duration              #from: 
    flen = tlen//2 + 1
    psd = spectrum(flen, delta_f, f_low)
    
    k, i = match(hp, sp, psd=psd, low_frequency_cutoff=f_low)  # k is the match and we need this
    snr=sigma(sp,psd=psd, low_frequency_cutoff=f_low)          # produce the snr for the deformed signal (could use the other but it would make no difference)
    disting=2*snr**2*(1-k)                                     #distinguishability

    print(spectrum)
    print('snr',snr)
    print('For m1 mass:', m1,'Deformation is:', d,'The match is: {:.4f}'.format(k))
    print('Distinguishability:',disting)
    print('---------------------------------------------------------------')


<function aLIGOaLIGOO3LowT1800545 at 0x7f43779338c0>
snr 26.80969042834891
For m1 mass: 1.36 Deformation is: 300 The match is: 0.9942
Distinguishability: 8.32638483658589
---------------------------------------------------------------
<function aLIGOaLIGO140MpcT1800545 at 0x7f4377933440>
snr 31.487259493996298
For m1 mass: 1.36 Deformation is: 300 The match is: 0.9898
Distinguishability: 20.26231383998462
---------------------------------------------------------------
<function aLIGOaLIGO175MpcT1800545 at 0x7f43779335f0>
snr 39.591315902113216
For m1 mass: 1.36 Deformation is: 300 The match is: 0.9885
Distinguishability: 36.058030174314574
---------------------------------------------------------------
<function EinsteinTelescopeP1600143 at 0x7f43779ab050>
snr 406.55888959428756
For m1 mass: 1.36 Deformation is: 300 The match is: 0.9890
Distinguishability: 3631.601656694414
---------------------------------------------------------------
